# ReactiveX under trio

# *Status :-|*
* `trio` can feed a ReactiveX pipeline, if that pipeline has no time-aware components
* With time-aware pipeline components, using the `asyncio` support in the notebook and ReactiveX gives unreliable results as I have it here. More development is indicated.

In [ ]:
#%pip install -U rx trio

In [ ]:
import rx
from rx import operators as op
import trio

In [ ]:
%autoawait trio
#%autoawait asyncio

In [ ]:
import ipywidgets as widgets
from IPython.display import display

In [ ]:
out = widgets.Output(layout={
    'border': '1px solid black',
    'height': '200px',
    'overflow_y': 'auto',
})

## Feed a ReactiveX pipeline from a Trio task

### First try a pipeline that's just a pure dropthrough, with no use of the scheduler

In [ ]:
s = rx.subject.Subject()
s.subscribe(lambda i: out.append_stdout(f" <{i}>"))
display(out)
out.clear_output()

async def char_by_char(string, delay=0.2):
    for c in string:
        await trio.sleep(delay)
        #out.append_stdout(" " + c)
        s.on_next(c)

async def parent():
    print("parent: started!")
    async with trio.open_nursery() as nursery:
        nursery.start_soon(char_by_char, "The quick brown fox", 0.1)
        nursery.start_soon(char_by_char, "jumped over the lazy dog", 0.3)
        print("parent: waiting for children to finish...")
        # -- we exit the nursery block here --
    print("parent: all done!")

trio.run(parent)

### Result: Yes, we can feed a ReactiveX pipeline from a trio task
***FIXME:*** Test time-delay operators, once we know how to make them work

In [ ]:
%time
trio.run(parent) 

## Test ReactiveX using the notebook's `asyncio` event loop

Trio and asyncio are based on coroutines. ReactiveX uses a scheduler. The package includes one to run in an asyncio loop. Maybe that one will work okay under trio. We test this proposition.

In [ ]:
%gui fui

In [ ]:
#%autoawait trio
%autoawait asyncio
%gui asyncio

In [ ]:
import asyncio

In [ ]:
from trioscheduler import TrioScheduler
from rx.scheduler.eventloop import AsyncIOThreadSafeScheduler
from rx.scheduler import TimeoutScheduler

In [ ]:
#shed = rx.scheduler.TrampolineScheduler()
#shed = TrioScheduler()
loop = asyncio.get_event_loop()
async_shed = AsyncIOThreadSafeScheduler(loop)
timeout_shed = TimeoutScheduler()
shed = async_shed
loop, shed

In [ ]:
def lowercase():
    def _lowercase(source):
        def subscribe(observer, scheduler = None):
            def on_next(value):
                observer.on_next(value.lower())

            return source.subscribe(
                on_next,
                observer.on_error,
                observer.on_completed,
                scheduler)
        return rx.create(subscribe)
    return _lowercase

rx.of("Alpha", "Beta", "Gamma", "Delta", "Epsilon").pipe(
        op.observe_on(shed),
        lowercase(),
        op.delay(1)
     ).subscribe(lambda value: print("Received {0}".format(value)), scheduler=shed)

### Result: Wrong
* only works once under the asyncio scheduler, second time wedges the kernel
* using `%autoawait asyncio` instead of `%autoawait trio` gives same failure
* using `%gui asyncio` same hang on 2nd run

In [ ]:
###!jupyter nbconvert --to script rx-trio.ipynb